# Conference scheduling problem

In [1]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

## Problem Statement

Recently Slalom Seattle was planning it's annual Innovation Symposium, and I was on the planning committee.

The planning team wanted to assign each individual to a schedule. Attendees had been asked to rank their preference for attending each topic. The problem was a large number of schedules would have to manually be created; over 600 attendees, needed to be assigned to 7 different topics across 3 sessions. The original plan was to whiteboard it and manually create schedules and assign people to schedules, an excruciating process that may have to be rerun as new responses or constraints came in. It also seemed difficult to ensure the best solution: how do you even pick the best subset of possible schedules?

There were also some constraints about the capacity of available rooms, more popular sessions could be put in a larger room, and we wanted to avoid having rooms half empty in one session and then bursting at the seams in a later session.

## Data Prep

### Let's have a look at the survey data

In [2]:
# Read data
file='data/selections.csv'
df = pd.read_csv(file)
df.head(5)

,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5,Choice 6,Choice 7
0,[Design Thinking] + [Lean] + [Agile],Modern Culture of Data,Intro to AI and ML,Internet of Things,Design Thinking for Experience & Service Design,Cloud Enablement 201,Robotic Process Automation (RPA)
1,[Design Thinking] + [Lean] + [Agile],Design Thinking for Experience & Service Design,Cloud Enablement 201,Modern Culture of Data,Internet of Things,Intro to AI and ML,Robotic Process Automation (RPA)
2,Robotic Process Automation (RPA),Design Thinking for Experience & Service Design,Intro to AI and ML,Internet of Things,Modern Culture of Data,Cloud Enablement 201,[Design Thinking] + [Lean] + [Agile]
3,Design Thinking for Experience & Service Design,[Design Thinking] + [Lean] + [Agile],Intro to AI and ML,Modern Culture of Data,Cloud Enablement 201,Robotic Process Automation (RPA),Internet of Things
4,Cloud Enablement 201,Intro to AI and ML,Internet of Things,Modern Culture of Data,[Design Thinking] + [Lean] + [Agile],Design Thinking for Experience & Service Design,Robotic Process Automation (RPA)


Each row represents an attendee, and the columns are their ranked preference for each topic. Choice 1 was their highest preference and Choice 7 was their lowest.

In [3]:
df.describe()

,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5,Choice 6,Choice 7
count,405,404,404,397,396,396,396
unique,7,7,7,7,7,7,7
top,[Design Thinking] + [Lean] + [Agile],Modern Culture of Data,Design Thinking for Experience & Service Design,Cloud Enablement 201,Internet of Things,Intro to AI and ML,Robotic Process Automation (RPA)
freq,84,70,68,68,66,72,86


Looks like there are a number of missing values where someone either hasn't provided a ranking (but they've confirmed they're attending) or they have only provided a top 1,2,3... We'll have to handle all of these cases.

In [4]:
# Get total number of respondants
N_total = len(df)
print(f'Total number of responses: {N_total}')

# Create a dictionary of the topics for later use
topics = np.unique(df.dropna().values)
topic_map = dict(zip(topics,range(len(topics))))

Total number of responses: 578


### Convert the data into a convenient format

The data format isn't ideal; we'd like to have topics as the columns and rankings as integer values in the table.

Let's convert this into a more convenient shape.

In [5]:
# Convert to ranked format
df = df.reset_index().melt(id_vars='index').dropna().pivot(columns='value',index='index', values='variable') # Index = the id of a particular response
df = df.reindex(pd.RangeIndex(0,N_total)) # Reindex becuase the previous step dropped all the rows where respondant didn't provide any preferences at all

In [6]:
# Convert choice rankings to integers
def choice_to_int(choice):
    try:
        return int(choice.split(' ')[-1])
    except:
        return np.nan

choice_to_int('Choice 2'), choice_to_int(np.nan)

(2, nan)

In [7]:
# Use applymap to clean every choice
df = df.applymap(choice_to_int)

We also want to fill missing preferences; we'll assume an individual's preference for topics not ranked is uniform but lower than those they have ranked.

In [8]:
# Add one to the max choice for each row
unranked_preference = df.max(axis=1).fillna(0) +1

# The column by column replace the missing values with the max per row
for col in df.columns:
    df[col] = df[col].fillna(value=unranked_preference)
df.sample(5, random_state=1990)

value,Cloud Enablement 201,Design Thinking for Experience & Service Design,Internet of Things,Intro to AI and ML,Modern Culture of Data,Robotic Process Automation (RPA),[Design Thinking] + [Lean] + [Agile]
410,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50,5.0,2.0,7.0,6.0,3.0,4.0,1.0
146,1.0,3.0,7.0,4.0,2.0,5.0,6.0
315,7.0,2.0,3.0,6.0,4.0,5.0,1.0
576,6.0,3.0,4.0,5.0,2.0,7.0,1.0


You can see the response at index `410` didn't provide any choice so every topic has a rank 1.

## Solve Using Linear Programming

We can restate this as a linear programming problem where we have various constraints on the number of attendees in each session, and the number of sessions and topics a particular person can attend. 

### Assign preference cost
We want to assign costs for assigning a particular person to a session, that way we can compute a total of cost of a configuration, and optimize. We can say the cost is zero for assigning them to one of their top 3 sessions, with an increasing cost if we assign them to a lower preference session.

Depending on the problem and constraints we might choose different values here. For example if we want to avoid assigning any non-top 3 choices at all we could use a higher value for Choice 4 and on.

In [9]:
cost_array = [0,0,0,2,3,4,5]
cost_remap = {i:cost_array[i-1] for i in range(1,len(cost_array)+1)}
print(f'Costs by choice: {cost_remap}')

# Remap values in dataframe
df = df.replace(cost_remap).astype(np.int8) # apply our map, and convert to int
df.sample(5, random_state=1990)

Costs by choice: {1: 0, 2: 0, 3: 0, 4: 2, 5: 3, 6: 4, 7: 5}


value,Cloud Enablement 201,Design Thinking for Experience & Service Design,Internet of Things,Intro to AI and ML,Modern Culture of Data,Robotic Process Automation (RPA),[Design Thinking] + [Lean] + [Agile]
410,0,0,0,0,0,0,0
50,3,0,5,4,0,2,0
146,0,0,5,2,0,3,4
315,5,0,0,4,2,3,0
576,4,0,2,3,0,5,0


The response at index `410` which didn't provide any preferences has a zero cost; it doesn't matter which session they are assigned to. 

### Convert to a 3D cost matrix
We need to assign a cost for allocating each attendee to a topic in each session. There are three sessions and the cost should be the same for a particular attendee to attend a topic regardless of the order they attend. So we need to transform our 2D array of costs into a 3D matrix.

In [10]:
cost_matrix = np.dot(np.expand_dims(df.values,-1), np.array([[1,1,1]]))
print(f'First dataframe row:\n{df.iloc[0]}', f'Corresponding item from cost array:\n{cost_matrix[0]}', f'Shape of full array: {cost_matrix.shape}', sep='\n\n')

First dataframe row:
value
Cloud Enablement 201                               4
Design Thinking for Experience & Service Design    3
Internet of Things                                 2
Intro to AI and ML                                 0
Modern Culture of Data                             0
Robotic Process Automation (RPA)                   5
[Design Thinking] + [Lean] + [Agile]               0
Name: 0, dtype: int8

Corresponding item from cost array:
[[4 4 4]
 [3 3 3]
 [2 2 2]
 [0 0 0]
 [0 0 0]
 [5 5 5]
 [0 0 0]]

Shape of full array: (578, 7, 3)


The resulting array has a shape (`Number of respondants`, `Number of Topics`, `Number of Sessions`) = (`578`,`7`,`3`). So we have 12,138‬ constraints to solve... that's a bit more than [Excel's Solver package](https://www.excel-easy.com/data-analysis/solver.html) limit of 200.

### Mixed Integer Programming from Google's Operations Reaserch package to the rescue

Google provides a fast and open source library for solving various combinatorial optimization problems: [Google OR tools](https://developers.google.com/optimization/)([GitHub](https://github.com/google/or-tools)). There are a few ways we could solve this but we'll use the [Mixed Integer Programming (MIP) package](https://developers.google.com/optimization/assignment/assignment_mip).

First we need to provide a room capacity, we can adjust these numbers iteratively to best fit sessions into rooms. The iteration process went as follows: assign room capacities run solver, evaluate variance in attendance across sessions and adjust session capacities accordingly.

In [11]:
# Set max capacity per room
topic_capacities = [85,85,85,85,90,80,75] # 578 people and 7 topics = we need an average of 82 people (578/7) in each session
list(zip(topics,topic_capacities))

[('Cloud Enablement 201 ', 85),
 ('Design Thinking for Experience & Service Design', 85),
 ('Internet of Things', 85),
 ('Intro to AI and ML', 85),
 ('Modern Culture of Data ', 90),
 ('Robotic Process Automation (RPA)', 80),
 ('[Design Thinking] + [Lean] + [Agile]', 75)]

### Create a model and define contraints

In [12]:
## Create solver model
solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                            pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Create assignment array
# A boolean (True/False) value for each possible attendee, topic, session combination
# We'll store these in a Numpy array of object datatype for easy indexing
# By summing across dimesions of the array we can view and enforce constraints
# In Python True is evaluated as 1 in a summation and False is evaluated as 0
x = np.zeros((N_total,7,3), dtype=object)
for attendee in range(N_total):
    for topic in range(7):
        for session in range(3):
            x[attendee, topic, session] = solver.BoolVar(f'x[{attendee}, {topic}, {session}]')

## Add constraints to the model
# Each attendee attends exactly one topic per session
# Therefore the sum across topics for each attendee, session is 1
for a in range(N_total):
    for s in range(3):
        solver.Add(solver.Sum(x[a,t,s] for t in range(7)) == 1)

# Each attendee attends each topic no more than 1 time
# For each attendee, topic the sum can be 1 or 0
for a in range(N_total):
    for t in range(7):
        solver.Add(solver.Sum(x[a,t,s] for s in range(3)) <= 1)  

# Max people per session
# Total count of attendees for each session to be less than capacities defined
for s in range(3):
    for t in range(7):
        solver.Add(solver.Sum(x[a,t,s] for a in range(N_total)) <= topic_capacities[t])
    
solver.Minimize(solver.Sum([cost_matrix[a,t,s] * x[a,t,s] for a in range(N_total) for t in range(7) for s in range(3)]))

In [13]:
# Run solver
sol = solver.Solve()

In [14]:
# Return resulting solution to a new array
x_sol = np.full_like(x, 0, dtype=int)
for a in range(N_total):
    for t in range(7):
        for s in range(3):
            x_sol[a,t,s] = x[a,t,s].solution_value()

## Evaluating the Solution

### Did our solver find an optimal solution?

In [15]:
sol == solver.OPTIMAL

True

In this case it did, great! If we tighten the constraints this might not always be true. For example reducing the room capacities.

What about the total cost:

In [16]:
solver.Objective().Value()

0.0

Great this means everyone gets to attend their top 3 sessions.

### How many attendees in each session?
We want to check that there isn't significant varaince across sessions so that we avoid half empty rooms in some sessions.

In [17]:
print("Total numbers in each session")
pd.DataFrame(x_sol.sum(axis=0),index=topics,columns=['S1','S2','S3'], dtype=int)

Total numbers in each session


,S1,S2,S3
Cloud Enablement 201,85,82,85
Design Thinking for Experience & Service Design,82,85,85
Internet of Things,85,84,78
Intro to AI and ML,85,85,85
Modern Culture of Data,90,90,90
Robotic Process Automation (RPA),76,77,80
[Design Thinking] + [Lean] + [Agile],75,75,75


## Save resulting conference schedules

In [18]:
schedule = np.argmax(x_sol,axis=1)
schedule_text = topics[schedule]
sched_df = pd.DataFrame(schedule_text, columns=[f'Session {i}' for i in range(1,4)])
sched_df.sample(5, random_state=1990)

,Session 1,Session 2,Session 3
410,Cloud Enablement 201,Intro to AI and ML,Robotic Process Automation (RPA)
50,Modern Culture of Data,[Design Thinking] + [Lean] + [Agile],Design Thinking for Experience & Service Design
146,Modern Culture of Data,Cloud Enablement 201,Design Thinking for Experience & Service Design
315,[Design Thinking] + [Lean] + [Agile],Internet of Things,Design Thinking for Experience & Service Design
576,Design Thinking for Experience & Service Design,Modern Culture of Data,[Design Thinking] + [Lean] + [Agile]


In [19]:
sched_df.to_csv('data/conference_schedule.csv',index=True)